# Tutorial 1: Single-Target SPY Prediction with Professional Benchmarking

**Learning Objectives:**
- Understand the fundamentals of quantitative trading strategy development
- Learn walk-forward backtesting to avoid look-ahead bias
- Implement feature engineering with sector ETFs
- **NEW: Professional benchmarking with information ratios and excess returns**
- **NEW: Generate publication-quality PDF tear sheets**
- Use xarray for standardized results handling
- Calculate risk-adjusted performance metrics with extended data coverage (15+ years)

**Blue Water Macro Corp Educational Framework © 2025**

## Part 1: Setup and Data Loading

First, let's import our libraries and understand what we're trying to accomplish.

In [ ]:
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import xarray as xr
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns

# Import our custom utilities
from utils_simulate import (
    simplify_teos, log_returns, p_by_year, 
    create_results_xarray, plot_xarray_results,
    calculate_performance_metrics, get_educational_help
)

# NEW: Import professional benchmarking and simulation framework
from single_target_simulator import (
    load_and_prepare_data, Simulate, 
    SingleTargetBenchmarkManager, SingleTargetBenchmarkConfig,
    sim_stats_single_target, L_func_2, L_func_3, L_func_4
)
from plotting_utils import create_professional_tear_sheet

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📚 Welcome to the Blue Water Macro Quantitative Trading Tutorial!")
print("🎯 Goal: Predict SPY returns using sector ETF data with professional benchmarking")
print("🆕 NEW: Extended data coverage (2010-present) and PDF tear sheets!")

### Educational Moment: Why Log Returns?

Before we dive into data loading, let's understand a fundamental concept in quantitative finance:

In [ ]:
# Get educational explanation
get_educational_help('log_returns')

### Load Market Data

We'll use SPDR sector ETFs as features to predict SPY (S&P 500) returns:

In [ ]:
# Professional Configuration Setup
config = {
    "target_etf": "SPY",
    "feature_etfs": ['XLK', 'XLF', 'XLV', 'XLY', 'XLP', 'XLE', 'XLI', 'XLB', 'XLU'],
    "start_date": "2010-01-01",  # Extended coverage: 15+ years
    "window_size": 400,
    "window_type": "expanding",
    "author": "Student"
}

# Use professional data loading
print("📥 Loading data with extended coverage (2010-present)...")
X, y = load_and_prepare_data(
    config["feature_etfs"] + [config["target_etf"]], 
    config["target_etf"], 
    start_date=config["start_date"]
)

print(f"✅ Loaded {len(X)} days of data for {len(config['feature_etfs'])} features")
print(f"📊 Date range: {X.index.min()} to {X.index.max()}")
print(f"🎯 Target: {config['target_etf']}")
print(f"📈 Features: {', '.join(config['feature_etfs'])}")

# Quick data overview
print(f"\n📊 Data Summary:")
print(f"   Features shape: {X.shape}")
print(f"   Target shape: {y.shape}")
print(f"   Missing values: {X.isna().sum().sum()} features, {y.isna().sum()} target")

## Part 2: Feature Engineering and Exploration

Let's convert prices to log returns and explore the relationships between sector ETFs and SPY:

In [ ]:
# Setup Professional Benchmarking Framework
benchmark_config = SingleTargetBenchmarkConfig(
    include_transaction_costs=True,
    rebalancing_frequency='daily'
)

benchmark_manager = SingleTargetBenchmarkManager(
    target_etf=config["target_etf"],
    feature_etfs=config["feature_etfs"],
    config=benchmark_config
)

print("🎯 Professional Benchmarking Configured!")
print(f"   Available benchmarks: {list(benchmark_manager.benchmarks.keys())}")
print(f"   Target ETF: {config['target_etf']}")
print(f"   Benchmark types:")
for name, benchmark in benchmark_manager.benchmarks.items():
    print(f"     - {name}: {benchmark.get_description()}")

## NEW: Professional Benchmarking Setup

One of the most important aspects of quantitative finance is comparing your strategy against appropriate benchmarks. Let's set up professional benchmarking:

In [ ]:
# Calculate log returns
returns = log_returns(prices).dropna()

# Separate features and target
X_features = returns[FEATURE_ETFS]
y_target = returns[TARGET_ETF]

print(f"📈 Features shape: {X_features.shape}")
print(f"🎯 Target shape: {y_target.shape}")

# Quick visualization
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Plot cumulative returns
(1 + returns).cumprod().plot(ax=ax1, alpha=0.7)
ax1.set_title('Cumulative Returns: SPY vs Sector ETFs')
ax1.set_ylabel('Cumulative Return')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Plot rolling correlation with SPY
rolling_corr = X_features.rolling(252).corr(y_target).dropna()
rolling_corr.plot(ax=ax2, alpha=0.8)
ax2.set_title('Rolling 1-Year Correlation with SPY')
ax2.set_ylabel('Correlation')
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

### Feature Analysis by Year

Let's analyze how the predictive power of each sector changes over time:

In [ ]:
# Analyze feature importance by year
print("🔍 Analyzing feature importance by year...")
yearly_correlations = p_by_year(X_features, y_target)

# Create heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(yearly_correlations, annot=True, cmap='RdYlBu_r', center=0, 
           fmt='.3f', cbar_kws={'label': 'Pearson Correlation'})
plt.title('Annual Feature Correlations with SPY Returns')
plt.xlabel('Year')
plt.ylabel('Sector ETF')
plt.tight_layout()
plt.show()

# Find most stable predictors
mean_abs_corr = yearly_correlations.abs().mean(axis=1).sort_values(ascending=False)
print("\n🏆 Most consistent predictors (by average absolute correlation):")
for etf, corr in mean_abs_corr.head(5).items():
    print(f"  {etf}: {corr:.3f}")

# Professional Walk-Forward Simulation with Multiple Position Strategies
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from utils_simulate import EWMTransformer

# Define position sizing strategies to test
position_strategies = [
    ('Binary', L_func_2),      # Simple binary: +1 or -1
    ('Quartile', L_func_3),    # Quartile-based: 0, 0.5, 1.5, 2.0
    ('Proportional', L_func_4) # Proportional to prediction strength
]

print("🚀 Running professional simulation with multiple strategies...")
print(f"📅 Period: {X.index.min()} to {X.index.max()}")
print(f"🔄 Position strategies: {[name for name, _ in position_strategies]}")

# Run simulations
regout_list = []
sweep_tags = []

for pos_name, pos_func in position_strategies:
    print(f"\n🎯 Running {pos_name} strategy...")
    
    # Create ML pipeline with exponential smoothing
    pipe = Pipeline([
        ('ewm', EWMTransformer(halflife=4)),
        ('ridge', Ridge(alpha=1.0))
    ])
    
    # Run simulation
    regout = Simulate(
        X, y, 
        window_size=config["window_size"],
        window_type=config["window_type"],
        pipe_steps=pipe,
        L_func=pos_func,
        tag=f"ridge_ewm4_{pos_name.lower()}"
    )
    
    regout_list.append(regout)
    sweep_tags.append(f"ridge_ewm4_{pos_name.lower()}")
    
    print(f"   ✅ Completed {len(regout)} predictions")

print("\n✅ All simulations completed!")
print(f"📊 Total strategies: {len(regout_list)}")
print(f"📈 Predictions per strategy: {len(regout_list[0])}")

In [ ]:
# Educational explanation
get_educational_help('walk_forward')

In [ ]:
# Professional Performance Analysis with Benchmarking
import time

print("📊 Calculating comprehensive performance statistics with benchmarking...")

# Define time range for analysis
trange = slice(regout_list[0].index[0], regout_list[0].index[-1])

# Calculate professional statistics with benchmarking
stats_df, enhanced_results = sim_stats_single_target(
    regout_list, 
    sweep_tags,
    author=config["author"],
    trange=trange,
    target_etf=config["target_etf"],
    feature_etfs=config["feature_etfs"],
    benchmark_manager=benchmark_manager,
    config=config
)

print("\n🏆 PROFESSIONAL PERFORMANCE SUMMARY")
print("=" * 60)
print(stats_df.round(4))

# Highlight key metrics
print("\n🎯 KEY INSIGHTS:")
best_strategy = stats_df.loc['sharpe'].idxmax()
best_sharpe = stats_df.loc['sharpe', best_strategy]
best_benchmark = stats_df.loc['best_benchmark', best_strategy]
excess_return = stats_df.loc['best_excess_return', best_strategy]
info_ratio = stats_df.loc['best_info_ratio', best_strategy]

print(f"   📈 Best Strategy: {best_strategy} (Sharpe: {best_sharpe:.3f})")
print(f"   🎯 Best Benchmark: {best_benchmark}")
print(f"   💰 Excess Return: {excess_return:.2%} annually")
print(f"   📊 Information Ratio: {info_ratio:.3f}")

# Show benchmark comparison for all strategies
print(f"\n📊 BENCHMARK ANALYSIS:")
for strategy in sweep_tags:
    benchmark = stats_df.loc['best_benchmark', strategy]
    excess = stats_df.loc['best_excess_return', strategy] 
    ir = stats_df.loc['best_info_ratio', strategy]
    print(f"   {strategy}: vs {benchmark} | Excess: {excess:.2%} | IR: {ir:.3f}")

print(f"\n📅 Analysis Period: {stats_df.loc['start_date', sweep_tags[0]]} to {stats_df.loc['end_date', sweep_tags[0]]}")

## Part 4: Results Analysis with xarray

Let's use xarray to analyze our results in a standardized way:

In [ ]:
# Convert results to xarray Dataset
results_df = pd.DataFrame(simulation_results)
results_df.set_index('dates', inplace=True)

# Create xarray dataset
results_xr = create_results_xarray({
    'strategy_returns': results_df['returns'],
    'spy_returns': results_df['actuals'],
    'predictions': results_df['predictions'],
    'positions': results_df['positions']
}, time_index=results_df.index)

print("📊 Results stored in xarray Dataset:")
print(results_xr)

# Calculate performance metrics
strategy_metrics = calculate_performance_metrics(results_xr.strategy_returns)
spy_metrics = calculate_performance_metrics(results_xr.spy_returns)

print("\n🏆 Performance Comparison:")
comparison_df = pd.DataFrame({
    'Strategy': strategy_metrics,
    'SPY Buy-Hold': spy_metrics
})
print(comparison_df.round(4))

In [ ]:
# Generate Professional PDF Tear Sheet
config['run_timestamp'] = time.strftime('%Y%m%d_%H%M%S')

print("📄 Generating publication-quality PDF tear sheet...")
pdf_path = create_professional_tear_sheet(
    list(enhanced_results.values()),  # Use enhanced results with benchmark data
    sweep_tags,
    config
)

print(f"✅ Professional tear sheet generated!")
print(f"📄 PDF: {pdf_path}")
print(f"📁 Location: {os.path.abspath(pdf_path)}")

# Also create xarray dataset for further analysis
results_xr = create_results_xarray(enhanced_results, time_index=enhanced_results[sweep_tags[0]].index)
print(f"\n📊 Results also available in xarray format:")
print(f"   Dimensions: {dict(results_xr.dims)}")
print(f"   Variables: {list(results_xr.data_vars)}")

# Quick performance visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Extract strategy returns for all strategies
strategy_returns = {}
for i, tag in enumerate(sweep_tags):
    strategy_returns[tag] = enhanced_results[tag]['perf_ret']

# 1. Cumulative returns comparison
for tag, returns in strategy_returns.items():
    cumret = (1 + returns).cumprod()
    cumret.plot(ax=axes[0,0], label=tag.replace('ridge_ewm4_', '').title(), alpha=0.8)

axes[0,0].set_title('Cumulative Strategy Returns')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Strategy vs SPY comparison (best strategy)
best_returns = enhanced_results[best_strategy]['perf_ret']
spy_returns = enhanced_results[best_strategy]['actual']

(1 + best_returns).cumprod().plot(ax=axes[0,1], label=f'Best Strategy ({best_strategy})', color='blue')
(1 + spy_returns).cumprod().plot(ax=axes[0,1], label='SPY Buy-Hold', color='red')
axes[0,1].set_title('Best Strategy vs SPY')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Risk-Return scatter
returns_data = []
vol_data = []
labels = []
for tag, returns in strategy_returns.items():
    returns_data.append(returns.mean() * 252)  # Annualized
    vol_data.append(returns.std() * np.sqrt(252))  # Annualized
    labels.append(tag.replace('ridge_ewm4_', '').title())

axes[1,0].scatter(vol_data, returns_data, s=100, alpha=0.7)
for i, label in enumerate(labels):
    axes[1,0].annotate(label, (vol_data[i], returns_data[i]), xytext=(5, 5), 
                      textcoords='offset points', fontsize=9)
axes[1,0].set_xlabel('Annualized Volatility')
axes[1,0].set_ylabel('Annualized Return')
axes[1,0].set_title('Risk-Return Profile')
axes[1,0].grid(True, alpha=0.3)

# 4. Benchmark excess returns
excess_returns = []
info_ratios = []
strategy_names = []
for tag in sweep_tags:
    excess_returns.append(stats_df.loc['best_excess_return', tag])
    info_ratios.append(stats_df.loc['best_info_ratio', tag])
    strategy_names.append(tag.replace('ridge_ewm4_', '').title())

bars = axes[1,1].bar(strategy_names, excess_returns, alpha=0.7)
axes[1,1].set_title('Excess Returns vs Best Benchmark')
axes[1,1].set_ylabel('Excess Return (%)')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].grid(True, alpha=0.3)

# Add information ratios as text
for i, (bar, ir) in enumerate(zip(bars, info_ratios)):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height + 0.001,
                  f'IR: {ir:.2f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print(f"\n🎉 Tutorial complete! You've learned professional quantitative finance techniques:")

## NEW: Publication-Quality PDF Tear Sheet

Let's generate a professional PDF tear sheet with our benchmark analysis:

### Visualization and Analysis

In [ ]:
# Create comprehensive performance plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Cumulative returns
strategy_cumret = (1 + results_xr.strategy_returns).cumprod()
spy_cumret = (1 + results_xr.spy_returns).cumprod()

strategy_cumret.plot(ax=axes[0,0], label='Strategy', color='blue')
spy_cumret.plot(ax=axes[0,0], label='SPY Buy-Hold', color='red')
axes[0,0].set_title('Cumulative Returns')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Rolling Sharpe ratio (252-day)
rolling_sharpe = (results_xr.strategy_returns.rolling(time=252).mean() / 
                 results_xr.strategy_returns.rolling(time=252).std() * np.sqrt(252))
rolling_sharpe.plot(ax=axes[0,1], color='green')
axes[0,1].set_title('Rolling 1-Year Sharpe Ratio')
axes[0,1].axhline(y=1.0, color='black', linestyle='--', alpha=0.5)
axes[0,1].grid(True, alpha=0.3)

# 3. Drawdown analysis
running_max = strategy_cumret.expanding(dim='time').max()
drawdown = (strategy_cumret - running_max) / running_max
drawdown.plot(ax=axes[1,0], color='red')
axes[1,0].fill_between(drawdown.time, drawdown.values, 0, alpha=0.3, color='red')
axes[1,0].set_title('Strategy Drawdown')
axes[1,0].set_ylabel('Drawdown %')
axes[1,0].grid(True, alpha=0.3)

# 4. Prediction vs actual scatter
axes[1,1].scatter(results_xr.predictions, results_xr.spy_returns, alpha=0.5)
axes[1,1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[1,1].axvline(x=0, color='black', linestyle='-', alpha=0.3)
axes[1,1].set_xlabel('Predictions')
axes[1,1].set_ylabel('Actual SPY Returns')
axes[1,1].set_title('Prediction Accuracy')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate prediction accuracy metrics
predictions = results_xr.predictions.values
actuals = results_xr.spy_returns.values

# Direction accuracy
direction_accuracy = np.mean(np.sign(predictions) == np.sign(actuals))
correlation = np.corrcoef(predictions, actuals)[0,1]

print(f"\n🎯 Prediction Metrics:")
print(f"   Direction Accuracy: {direction_accuracy:.1%}")
print(f"   Prediction-Actual Correlation: {correlation:.4f}")

## Part 5: Student Exercises

Now it's your turn to experiment and learn! Try these exercises to deepen your understanding:

### Exercise 1: Position Sizing Improvements

Modify the position sizing function to use prediction confidence:

In [ ]:
# TODO: Implement confidence-weighted position sizing
# Hint: Scale position size by absolute value of prediction

def confidence_weighted_positions(predictions, max_leverage=2.0):
    """
    Create position sizes based on prediction confidence.
    
    Your task:
    1. Calculate the absolute value of predictions (confidence)
    2. Normalize confidence to [0, max_leverage] range
    3. Apply the sign of the original prediction
    
    Returns:
        Array of position sizes
    """
    # YOUR CODE HERE
    pass

# Test your function
test_predictions = np.array([0.01, -0.02, 0.005, -0.03, 0.015])
test_positions = confidence_weighted_positions(test_predictions)
print(f"Predictions: {test_predictions}")
print(f"Positions: {test_positions}")

## Part 6: Key Takeaways

Congratulations! You've completed the enhanced single-target simulation tutorial with professional benchmarking. Here's what you learned:

### 🎓 Concepts Mastered:
1. **Extended Data Coverage**: Working with 15+ years of market data (2010-present)
2. **Professional Configuration**: Structured parameter management for reproducible research
3. **Walk-Forward Analysis**: Preventing look-ahead bias in backtests
4. **Professional Benchmarking**: Information ratios, excess returns, and best benchmark selection
5. **Multiple Position Strategies**: Binary, quartile, and proportional position sizing
6. **Publication-Quality Reports**: PDF tear sheets with comprehensive performance analysis
7. **xarray Integration**: Standardized handling of financial time series

### 🆕 NEW Professional Features:
- ✅ **Benchmark Framework**: Buy-and-hold, zero-return, and custom benchmarks
- ✅ **Information Ratios**: Risk-adjusted excess return measurement
- ✅ **PDF Tear Sheets**: Publication-quality performance reports
- ✅ **Extended Data**: 15+ years of robust backtesting data
- ✅ **Configuration Management**: Professional parameter handling

### 📊 Key Metrics You Now Understand:
- **Sharpe Ratio**: Risk-adjusted returns (return/volatility)
- **Information Ratio**: Excess return per unit of tracking error
- **Excess Return**: Strategy outperformance vs benchmark
- **Maximum Drawdown**: Largest peak-to-trough decline
- **Best Benchmark**: Optimal comparison for each strategy

### 🚀 Next Steps:
- Complete the updated exercises below to practice these concepts
- Move to Tutorial 2 for multi-target portfolio strategies with advanced benchmarking
- Experiment with different time periods and ETF universes
- Try implementing custom benchmarks and risk metrics

### 💡 Professional Applications:
- **Portfolio Management**: These techniques are used by quantitative portfolio managers
- **Risk Management**: Benchmark analysis is critical for institutional risk control  
- **Performance Attribution**: Understanding sources of returns vs market factors
- **Client Reporting**: PDF tear sheets are standard in professional asset management

### 📚 Additional Resources:
- [QuantNet Forums](https://quantnet.com): Connect with other quant students
- [Blue Water Macro Blog](https://bluewatermacro.com): Industry insights and research
- [xarray Documentation](https://xarray.pydata.org): Master multi-dimensional data analysis

**Ready for advanced multi-asset portfolio strategies? Proceed to Tutorial 2!**

In [ ]:
# TODO: Create momentum features
# Ideas:
# - 5-day, 20-day moving averages
# - RSI (Relative Strength Index)
# - Price momentum (current price vs N-day ago)

def create_momentum_features(prices, returns):
    """
    Create momentum-based features for prediction.
    
    Your task:
    1. Calculate short-term (5-day) and long-term (20-day) moving averages
    2. Create momentum indicators (e.g., current vs past prices)
    3. Add volatility measures (rolling standard deviation)
    
    Returns:
        DataFrame with momentum features
    """
    # YOUR CODE HERE
    pass

# Test with SPY data
# momentum_features = create_momentum_features(prices[TARGET_ETF], returns[TARGET_ETF])
# print(momentum_features.head())

### Exercise 3: Model Comparison

Compare different ML models using xarray:

In [ ]:
# TODO: Compare Ridge, Random Forest, and Linear Regression
# Use xarray to store results from multiple models
# Create performance comparison table

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

models = {
    'Ridge': Ridge(alpha=1.0),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'LinearRegression': LinearRegression()
}

# YOUR CODE HERE
# 1. Run simulation for each model
# 2. Store results in xarray with 'model' dimension
# 3. Compare performance metrics
# 4. Create visualization showing all models

print("🎯 Model comparison exercise - implement your solution above!")

## Part 6: Key Takeaways

Congratulations! You've completed the single-target simulation tutorial. Here's what you learned:

### 🎓 Concepts Mastered:
1. **Log Returns**: Why they're essential for financial modeling
2. **Walk-Forward Analysis**: Preventing look-ahead bias in backtests
3. **Feature Analysis**: Understanding predictor stability over time
4. **xarray Integration**: Standardized handling of financial time series
5. **Performance Metrics**: Risk-adjusted return measurement

### 🚀 Next Steps:
- Complete the exercises above to deepen your understanding
- Move to Tutorial 2 for multi-target portfolio strategies
- Experiment with different time periods and ETF universes
- Try implementing transaction costs and slippage

### 📚 Additional Resources:
- [QuantNet Forums](https://quantnet.com): Connect with other quant students
- [Blue Water Macro Blog](https://bluewatermacro.com): Industry insights and research
- [xarray Documentation](https://xarray.pydata.org): Master multi-dimensional data analysis

**Ready for more advanced techniques? Proceed to Tutorial 2: Multi-Target Portfolio Strategies!**